### Train a machine learning model 

#### Step 1: Setup 

In [105]:
#%pip install pandas numpy -qU

In [106]:
#%pip install scikit-learn -qU

In [107]:
# %pip install shap -qU

In [108]:
 #%pip install xgboost -qU

In [109]:
# %pip install  matplotlib -qU

In [110]:
 #%pip install numpy -qU

In [111]:
# %pip install catboost -qU

In [112]:
# %pip install lightgbm -qU

In [113]:
# %pip install seaborn -qU

In [114]:
# %pip install optuna -qU

In [115]:
# !nvidia-smi

In [116]:
import pandas as pd
import os
import sklearn
import shap
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_validate,RandomizedSearchCV, StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
from trainMLmodel import surfaceDesignTopAndBottom,selectDescriptors,combineRankAndDescriptorsClassyfication,explainer, combineRankAndDescriptorsClassificationbotisFLAT, combineRankAndDescriptorsClassyfication_Danilingus
import numpy as np

In [117]:
cd

/bme001.mnt/home/20212358


#### Step 2: load the data file

In [118]:
# Define the location of the csv files imageOutliersRemoved and Ranking
#pathToImageObject=os.getcwd()+"/DataAnalysis/"
pathToImageObject="/bme001.mnt/home/20212358"
locationOfImageObject1="cleaned_rawImagedataFI_final.csv"
locationOfImageObject2="DataAnalysis_withJan/Screen_Mean_Mean_FilteredNuclei_new_AreaShape_Area.csv"
# Define the location of the annotation files
pathToTopoUnitImages="/FeatureImages/FeatureImages/"
allTopoDescriptors=pd.read_csv("allTopoDescriptors.csv")
# check if file exists
if os.path.isfile(locationOfImageObject1):
    imagedataOutliersRemoved=pd.read_csv(locationOfImageObject1,low_memory=False)
    if os.path.isfile(locationOfImageObject2):
        imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)
    else:
        print("File not found, check if you finished ranking the features")
else:
    print('File not find, check if you removed the outliers')


In [119]:
#imagedataOutliersRemoved["Mean_Nephrin_Granularity_8_CorrCellBody"]

In [120]:
#replacing ranking by previous module with ranking sorted from lowest hit to the highest
#locationOfImageObject2=pathToImageObject+"/DataAnalysis/Screen_Mean_Mean_Nephrin_AreaShape_MajorAxisLength.csv"
# locationOfImageObject2=pathToImageObject+"/DataAnalysis/PostBlindTestHits/Screen_Mean_Mean_Nephrin_Granularity_8_CorrCellBody.csv"
# imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)

In [121]:
imageDataRankedSurfaces

FeatureIdx  Screen_ADpvalue_Mean_FilteredNuclei_new_AreaShape_Area  \
0            259                                               0.25        
1           1258                                               0.25        
2             68                                               0.25        
3            146                                               0.25        
4           1292                                               0.25        
...          ...                                                ...        
2024         629                                               0.25        
2025         943                                               0.25        
2026        1102                                               0.25        
2027        2122                                               0.25        
2028         531                                               0.25        

      Screen_KSpvalue_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                              0.783883        
1                                              0.798601        
2                                              0.575175        
3                                              0.637363        
4                                              0.850816        
...                                                 ...        
2024                                           0.212121        
2025                                           0.328283        
2026                                           0.575175        
2027                                           0.474026        
2028                                           0.433566        

      Screen_SNR_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                              4.710285   
1                                              3.716335   
2                                              3.109034   
3                                              4.196396   
4                                              2.659592   
...                                                 ...   
2024                                           6.420720   
2025                                           7.691701   
2026                                           8.752283   
2027                                           5.243277   
2028                                           7.228937   

      Screen_Mean_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                            985.010974    
1                                            955.062529    
2                                            923.971803    
3                                            914.961843    
4                                            914.821578    
...                                                 ...    
2024                                         623.940202    
2025                                         622.893170    
2026                                         619.887914    
2027                                         612.558254    
2028                                         599.825623    

      Screen_SD_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                           209.119197   
1                                           256.990462   
2                                           297.189389   
3                                           218.035177   
4                                           343.970654   
...                                                ...   
2024                                         97.176046   
2025                                         80.982497   
2026                                         70.825855   
2027                                        116.827379   
2028                                         82.975629   

      Screen_Median_Mean_FilteredNuclei_new_AreaShape_Area    Rank  
0                                            951.684211        1.0  
1                                            949.607143    

In [122]:
# locationOfImageObject3=pathToImageObject+"/DataAnalysis/Screen_Mean_Count_CellBody_new_Ranking.csv"
# imageDataRankedCount=pd.read_csv(locationOfImageObject3,low_memory=False)
# imageDataRankedCount

#### Step 3: Select Features for the machine learning.

##### Step 3.1 Vizualize the topographies design from top and bottom of the rank.

First, select the feature of interest

In [123]:
featureOfInterest='Screen_Mean_Mean_FilteredNuclei_new_AreaShape_Area'

Second, select the number of top and bottom ranked feautures you want to select

In [124]:
numberOfRankedFeaturesToVisualize=100

In [125]:
#surfaceDesignTopAndBottom(imageDataRankedSurfaces,pathToTopoUnitImages,featureOfInterest,number=numberOfRankedFeaturesToVisualize)
imagedataOutliersRemoved

AreaOccupied_AreaOccupied_CellBody_new  \
0                                    195441.0   
1                                    190993.0   
2                                    157466.0   
3                                    128276.0   
4                                    159374.0   
...                                       ...   
31915                                146459.0   
31916                                113561.0   
31917                                136466.0   
31918                                108352.0   
31919                                132796.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_new  \
0                                           41149.0   
1                                           34146.0   
2                                           21593.0   
3                                           30686.0   
4                                           36365.0   
...                                             ...   
31915                                       28944.0   
31916                                       21965.0   
31917                                       22229.0   
31918                                       18080.0   
31919                                       35925.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_target  \
0                                              38157.0   
1                                              33484.0   
2                                              19752.0   
3                                              28584.0   
4                                              35031.0   
...                                                ...   
31915                                          24587.0   
31916                                          19472.0   
31917                                          18386.0   
31918                                          12223.0   
31919                                          27669.0   

       AreaOccupied_AreaOccupied_MaskedNuclei  \
0                                    147590.0   
1                                    178264.0   
2                                    112684.0   
3                                    115514.0   
4                                    147877.0   
...                                       ...   
31915                                128723.0   
31916                                 96719.0   
31917                                109961.0   
31918                                 73657.0   
31919                                106458.0   

       AreaOccupied_AreaOccupied_Nephrin  AreaOccupied_Perimeter_CellBody_new  \
0                               166158.0                              17129.0   
1                               210194.0                              18661.0   
2                               133496.0                              13267.0   
3                               138323.0                              11218.0   
4                               200466.0                              16275.0   
...                                  ...                                  ...   
31915                           165853.0                              13291.0   
31916                           153661.0                              10378.0   
31917                           143216.0                              12602.0   
31918                           146310.0                              10716.0   
31919                           133671.0                              10817.0   

       AreaOccupied_Perimeter_FilteredNuclei_new  \
0                                         6242.0   
1                                         5837.0   
2                                         3797.0   
3                                         4632.0   
4                                         5380.0   
...                                          ...   
31915                                     4030.0   
31916                                     3448.0   
31917                         

### Step 4 Load and vizualize descriptors data

##### Step 4.1 select descriptors based on a keyword 

In [126]:
selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
             # keywordsToInclude=["Kamiel"],
                 # keywordsToExclude=["_Index","Height","Width", "Center","Texture","ImageNumber","Zernike","sd","Count","Number","var","AngleBetweenNeighbors","mad","Neighbors"])
#selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
              #  keywordsToInclude=["Pattern","Kamiel"],
                 #keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike"])
   keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike","Texture"])

In [127]:
pd.options.display.max_columns = 30
selectedDescriptors

FeatureIdx  Pattern_AreaOccupied_AreaOccupied_Pattern  \
0              1                                       5215   
1              2                                       2604   
2              3                                      20366   
3              4                                       4864   
4              5                                      17246   
...          ...                                        ...   
2171        2172                                       3821   
2172        2173                                      16244   
2173        2174                                       1855   
2174        2175                                       4794   
2175        2176                                      23548   

      Pattern_AreaOccupied_Perimeter_Pattern  \
0                                        330   
1                                        422   
2                                        910   
3                                        439   
4                                       1252   
...                                      ...   
2171                                     436   
2172                                    1497   
2173                                     306   
2174                                     492   
2175                                    1632   

      Pattern_AreaOccupied_TotalArea_Pattern  Pattern_Count_Pattern  \
0                                      40000                      1   
1                                      10000                      1   
2                                      40000                      1   
3                                      10000                      1   
4                                      40000                      1   
...                                      ...                    ...   
2171                                   10000                      1   
2172                                   78400                      2   
2173                                   10000                      1   
2174                                   10000                      2   
2175                                   78400                      1   

      Pattern_AreaShape_Area_sd  Pattern_AreaShape_Compactness_sd  \
0                         0.000                          0.000000   
1                         0.000                          0.000000   
2                         0.000                          0.000000   
3                         0.000                          0.000000   
4                         0.000                          0.000000   
...                         ...                               ...   
2171                      0.000                          0.000000   
2172                   3183.395                          0.654076   
2173                      0.000                          0.000000   
2174                   1681.500                          0.306309   
2175                      0.000                          0.000000   

      Pattern_AreaShape_Eccentricity_sd  Pattern_AreaShape_Extent_sd  \
0                              0.000000                     0.000000   
1                              0.000000                     0.000000   
2                              0.000000                     0.000000   
3                              0.000000                     0.000000   
4                              0.000000                     0.000000   
...                                 ...                          ...   
2171                           0.000000                     0.000000   
2172                           0.052437                     0.022692   
2173                           0.000000                     0.000000   
2174                           0.553171                     0.158168   
2175                           0.000000                     0.000000   

      Pattern_AreaShape_FormFactor_sd  Pattern_AreaShape_MajorAxisLength_sd  \
0                            0.000000         

In [128]:
Screen_Mean_Mean_Nephrin_Granularity_1 = pd.read_csv("Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody.csv")
Screen_Mean_Mean_Nephrin_Granularity_1

FeatureIdx  Screen_ADpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0           1658                                           0.250000         
1            189                                           0.083959         
2           1270                                           0.095665         
3            122                                           0.250000         
4           1353                                           0.250000         
...          ...                                                ...         
2086        1836                                           0.163023         
2087        1860                                           0.250000         
2088        1394                                           0.250000         
2089         757                                           0.069970         
2090        1292                                           0.250000         

      Screen_KSpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              0.730111         
1                                              0.126748         
2                                              0.055944         
3                                              0.647552         
4                                              0.700123         
...                                                 ...         
2086                                           0.078568         
2087                                           0.860428         
2088                                           0.730111         
2089                                           0.078568         
2090                                           0.893706         

      Screen_SNR_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              5.508653    
1                                              5.717872    
2                                              4.253026    
3                                              5.190216    
4                                              4.866214    
...                                                 ...    
2086                                           5.657096    
2087                                           5.332652    
2088                                           5.167565    
2089                                           6.501527    
2090                                           3.432248    

      Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                             46.026999     
1                                             45.273450     
2                                             45.259933     
3                                             45.068389     
4                                             44.444154     
...                                                 ...     
2086                                          30.269535     
2087                                          29.868709     
2088                                          29.775861     
2089                                          29.053096     
2090                                          28.751350     

      Screen_SD_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              8.355400   
1                                              7.917885   
2                                             10.641819   
3                                              8.683335   
4                                              9.133211   
...                                                 ...   
2086                                           5.350720   
2087                                           5.601099   
2088                                           5.762068   
2089                                           4.468657   
2090                                           8.376829   

      Screen_Median_Mean_Nephrin_Granularity_1_CorrCellBody    Rank  
0                                             45.283829         1.0

### Step 5: Train Machine Learning models

##### Step 5.1 : train the model based on the ranked surfaces and the descriptors selected in step 4.1

In [129]:
imageDataRankedSurfaces

FeatureIdx  Screen_ADpvalue_Mean_FilteredNuclei_new_AreaShape_Area  \
0            259                                               0.25        
1           1258                                               0.25        
2             68                                               0.25        
3            146                                               0.25        
4           1292                                               0.25        
...          ...                                                ...        
2024         629                                               0.25        
2025         943                                               0.25        
2026        1102                                               0.25        
2027        2122                                               0.25        
2028         531                                               0.25        

      Screen_KSpvalue_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                              0.783883        
1                                              0.798601        
2                                              0.575175        
3                                              0.637363        
4                                              0.850816        
...                                                 ...        
2024                                           0.212121        
2025                                           0.328283        
2026                                           0.575175        
2027                                           0.474026        
2028                                           0.433566        

      Screen_SNR_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                              4.710285   
1                                              3.716335   
2                                              3.109034   
3                                              4.196396   
4                                              2.659592   
...                                                 ...   
2024                                           6.420720   
2025                                           7.691701   
2026                                           8.752283   
2027                                           5.243277   
2028                                           7.228937   

      Screen_Mean_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                            985.010974    
1                                            955.062529    
2                                            923.971803    
3                                            914.961843    
4                                            914.821578    
...                                                 ...    
2024                                         623.940202    
2025                                         622.893170    
2026                                         619.887914    
2027                                         612.558254    
2028                                         599.825623    

      Screen_SD_Mean_FilteredNuclei_new_AreaShape_Area  \
0                                           209.119197   
1                                           256.990462   
2                                           297.189389   
3                                           218.035177   
4                                           343.970654   
...                                                ...   
2024                                         97.176046   
2025                                         80.982497   
2026                                         70.825855   
2027                                        116.827379   
2028                                         82.975629   

      Screen_Median_Mean_FilteredNuclei_new_AreaShape_Area    Rank  
0                                            951.684211        1.0  
1                                            949.607143    

In [130]:
imageDataRankedSurfaces.shape

(2029, 8)

In [131]:
# #pd.merge(left=imageDataRankedSurfaces, right=result_df,
#                       how='left', left_on="FeatureIdx", 
#                       right_on="FeatureIdx").Rank_x

In [132]:
X,Y=combineRankAndDescriptorsClassyfication(SortedRank=imageDataRankedSurfaces,Descriptors=selectedDescriptors,numberPerClass=89)

X.shape

(178, 383)

In [133]:
Y=Y.astype(bool)

In [134]:
def reduce_memory_usage(df):
    import gc
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()
                
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)

                elif str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)
                    
                if(df[col].nunique()>100):
                    df[col] = df[col].astype('category')
                else:
                    if((col_type != 'datetime64[ns]')):
                        df[col] = df[col].astype(str)
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage became: ",mem_usg," MB")
    gc.collect()
    return df

X=reduce_memory_usage(X)

Memory usage of dataframe is 0.52 MB
Memory usage became:  0.13405799865722656  MB


##### Step 5.2: split X and Y in training and test data set

In [135]:
a=0
b=0
for i in Y.values:
    if (i==False):
        b+=1
    else:
        a+=1
a,b

(89, 89)

In [136]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y )
# X_train,X_val,y_train,y_val=train_test_split(X_train,y_train)

In [137]:
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.2,stratify=y_test )


##### Step 5.3: select a classification model

In [138]:
import lightgbm as lgb
import joblib
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score


def objective(trial,X_train =X_train ,y_train = y_train, X_test = X_test,y_test=y_test, big_df = X, big_y=Y ):

    param = {
    'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart','rf']), #, 'rf' 
    'num_leaves': trial.suggest_int('num_leaves', 2, 2500),
    'max_depth':trial.suggest_int('max_depth', -10, 900),
    'objective':trial.suggest_categorical('objective', ['binary']),
    'learning_rate': trial.suggest_float('learning_rate', 1e-12, 0.99, log=False),
    'n_estimators': trial.suggest_int('n_estimators', 100, 24_100),
    #'subsample_for_bin': trial.suggest_int('subsample_for_bin', 50_000, 450_000, 50_000),
    'random_state': trial.suggest_categorical('random_state', [42]),
    'data_sample_strategy':trial.suggest_categorical('data_sample_strategy', ['bagging','goss']),
    # 'tree_learner':trial.suggest_categorical('tree_learner', ['serial', 'feature', 'data', 'voting']),
    'deterministic':trial.suggest_categorical('deterministic', [True]),
    'force_col_wise':trial.suggest_categorical('force_col_wise', [True,False]),
    # 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 2_230),
    
    'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf',  0, 100),
# #     'bagging_fraction': trial.suggest_float('bagging_fraction',  1e-12, 1),
    'feature_fraction': trial.suggest_float('feature_fraction', 1e-12, 0.9, log=False),
    'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 1e-12, 0.9, log=False),
    'extra_trees':trial.suggest_categorical('extra_trees', [True,False]),
    'lambda_l1': trial.suggest_float('lambda_l1',  0, 100),
    'lambda_l2': trial.suggest_float('lambda_l2',  0, 100),
    'linear_lambda': trial.suggest_float('linear_lambda',  0, 100),
    'drop_rate': trial.suggest_float('drop_rate',  0, 1),
    'max_drop': trial.suggest_int('max_drop', 0, 2000),
    'skip_drop': trial.suggest_float('skip_drop',  0, 1),
    'uniform_drop':trial.suggest_categorical('uniform_drop', [True,False]),
    'top_rate': trial.suggest_float('top_rate',  0, 0.5),
    'other_rate': trial.suggest_float('other_rate',  0, 0.5),
    'min_data_per_group': trial.suggest_int('min_data_per_group', 1, 2000),
    'max_cat_threshold': trial.suggest_int('max_cat_threshold', 1, 2000),
    'cat_l2': trial.suggest_float('cat_l2',  0, 100),
    'cat_smooth': trial.suggest_float('cat_smooth',  0, 100),
    'top_k': trial.suggest_int('top_k', 1, 2000),
    'cegb_tradeoff': trial.suggest_float('cegb_tradeoff',  0, 100),
    # 'cegb_penalty_split': trial.suggest_float('cegb_penalty_split',  0, 100),
    # 'path_smooth': trial.suggest_float('path_smooth',  0, 100),
    # 'num_grad_quant_bins': trial.suggest_int('num_grad_quant_bins', 1, 2000),
#     'is_unbalance':trial.suggest_categorical('uniform_drop', [True,False]),
    'scale_pos_weight': trial.suggest_float('scale_pos_weight',  1e-12, 100),
    'sigmoid': trial.suggest_float('sigmoid',  1e-12, 100),
    # 'boost_from_average':trial.suggest_categorical('boost_from_average', [True,False]),
    'max_bin':trial.suggest_int('max_bin', 50, 1_000),
        
    'n_jobs':trial.suggest_categorical('n_jobs', [-1]), 
    'verbose': trial.suggest_categorical('verbose', [-1]), 
    }


    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])


    y_pred_test = model.predict(X_test)
    # print(y_pred_test)
    

    #accuracy_test = balanced_accuracy_score(y_pred_test, y_test)

    #accuracy_test = (sklearn.model_selection.cross_val_score (model, big_df, big_y, cv=5, scoring='balanced_accuracy', n_jobs=1)).mean()
    # try:
    #     accuracy_test = roc_auc_score(y_pred_test, y_test)
    # except Exception:
    #     accuracy_test=0

    try:
        predicted_train = model.predict(X_train)
        probabilities_train = model.predict_proba(X_train)
        predicted_test = model.predict(X_test)
        probabilities_test = model.predict_proba(X_test)
            
        train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
        test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
    
        accuracy_train = metrics.auc(train_fpr, train_tpr)
        accuracy_test = metrics.auc(test_fpr, test_tpr)

        bal_accuracy_train = balanced_accuracy_score(predicted_train, y_train)
        bal_accuracy_test = balanced_accuracy_score(predicted_test, y_test) 
    except Exception:
        accuracy_test=0
        accuracy_train = 0
        bal_accuracy_test = 0
        bal_accuracy_train=0


    joblib.dump(study, 'N_Area_another_lgbm_with_val.pkl')

    if (int(trial.number)%50==0):
        print("BACKUP GOES BRRRRR")
        joblib.dump(study, 'N_Area_another_lgbm_with_val_BACKUP.pkl')


    return accuracy_test, accuracy_train,bal_accuracy_train,bal_accuracy_test

In [139]:
# https://vaex.io/docs/tutorial_ml.html#CatBoost-example

In [140]:
def logging_callback(study, frozen_trial):
    # Retrieve previous best values for both objectives
    previous_best_value_0 = study.user_attrs.get("previous_best_value_0", None)
    previous_best_value_1 = study.user_attrs.get("previous_best_value_1", None)
    previous_best_value_2 = study.user_attrs.get("previous_best_value_2", None)
    previous_best_value_3 = study.user_attrs.get("previous_best_value_3", None)

    
    # Get the current trial's values
    current_values = frozen_trial.values

    if current_values is None:
        return  # Skip if the trial doesn't have valid results

    # current_value_0, current_value_1 = current_values
    current_value_0, current_value_1,current_value_2,current_value_3 = current_values


    # Check if both objectives improved
    if (
        (previous_best_value_0 is None or current_value_0 >= previous_best_value_0) and
        (previous_best_value_1 is None or current_value_1 >= previous_best_value_1) and
        (previous_best_value_2 is None or current_value_2 >= previous_best_value_2) and
        (previous_best_value_3 is None or current_value_3 >= previous_best_value_3) 
    ):
        # Update the best values in the study's user attributes
        study.set_user_attr("previous_best_value_0", current_value_0)
        study.set_user_attr("previous_best_value_1", current_value_1)
        study.set_user_attr("previous_best_value_2", current_value_2)
        study.set_user_attr("previous_best_value_3", current_value_3)

        # Log the improved metrics
        print(40*"*",)
        print("\n\n\n")
        print(
            f"\nNew best trial with both objectives improved!\n"
            f"Trial {frozen_trial.number}:\n"
            f"  Objective 0: {current_value_0}\n"
            f"  Objective 1: {current_value_1}\n"
            f"  Objective 2: {current_value_2}\n"
            f"  Objective 3: {current_value_3}\n"
            f"  Params: {frozen_trial.params}\n"
        )
        print("\n\n\n")

        print(40*"*","\n")


In [141]:
#!rm N_Area_another_lgbm_with_val.pkl

In [142]:
!free -th

              total        used        free      shared  buff/cache   available
Mem:          503Gi        14Gi       400Gi       4.9Gi        87Gi       480Gi
Swap:         3.7Gi       3.7Gi       1.0Mi
Total:        507Gi        18Gi       400Gi


In [173]:
import shap
import matplotlib.pyplot as plt
import joblib
import lightgbm as lgb

# Load the Optuna study
study = joblib.load('another_lgbm_with_val.pkl')  # Ensure the correct path
plt.close('all')

# Retrieve the best trial from the Pareto front
best_trial = study.best_trials[2]  # Select the first Pareto-optimal trial
best_params = best_trial.params

print("Best Parameters:", best_params)

# Load the dataset (ensure these variables are defined in your notebook)
# Assuming X_train, y_train, X_test are already defined
# X_train = ...
# y_train = ...
# X_test = ...

# Train the LightGBM model with the best parameters
model = lgb.LGBMClassifier(**best_params)  # Use LGBMRegressor if regression
model.fit(X_train, y_train)

# Use TreeExplainer for LightGBM models
explainer = shap.TreeExplainer(model)

# Compute SHAP values
shap_values = explainer.shap_values(X_test)

# Plot and save the SHAP summary plot with Arial font and size 11
shap.summary_plot(shap_values, X_test, show=False)

# Get current axes
ax = plt.gca()

# Modify font properties for labels, title, and tick labels
ax.set_xlabel(ax.get_xlabel(), fontname='Arial', fontsize=11)
ax.set_ylabel(ax.get_ylabel(), fontname='Arial', fontsize=11)
ax.set_title(ax.get_title(), fontname='Arial', fontsize=11)
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontname('Arial')
    item.set_fontsize(11)

plt.savefig("shap_summary_plot_arial.png", bbox_inches='tight')
print("SHAP summary plot saved as 'shap_summary_plot_arial.png'")

Best Parameters: {'boosting_type': 'dart', 'num_leaves': 1183, 'max_depth': 799, 'objective': 'binary', 'learning_rate': 0.6837062610428098, 'n_estimators': 23827, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 26.34563463100433, 'feature_fraction': 0.758137890579718, 'feature_fraction_bynode': 0.5187965156833386, 'extra_trees': True, 'lambda_l1': 21.89754343655223, 'lambda_l2': 78.5045138370766, 'linear_lambda': 51.14797727541568, 'drop_rate': 0.980687406253138, 'max_drop': 1593, 'skip_drop': 0.4044587467381259, 'uniform_drop': False, 'top_rate': 0.4973967907584659, 'other_rate': 0.026200650319693167, 'min_data_per_group': 251, 'max_cat_threshold': 337, 'cat_l2': 70.22737946408505, 'cat_smooth': 91.46445306394386, 'top_k': 1224, 'cegb_tradeoff': 10.12911758309378, 'scale_pos_weight': 27.872893346379175, 'sigmoid': 92.16115148351307, 'max_bin': 574, 'n_jobs': -1, 'verbose': -1}


/home/20212358/.local/lib/python3.10/site-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


SHAP summary plot saved as 'shap_summary_plot_arial.png'


In [174]:
import joblib
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier, plot_importance

# Load Optuna study
study = joblib.load('another_lgbm_with_val.pkl')
plt.close('all')

# Get the best trial parameters
best_trial = study.best_trials[2]
best_params = best_trial.params

# Train the model with the best parameters
bestClassyficationModel = LGBMClassifier(**best_params)
bestClassyficationModel.fit(X_train, y_train)

# Plot feature importance with Arial font and size 11
fig, ax = plt.subplots(figsize=(10, 10))
plot_importance(bestClassyficationModel, max_num_features=15, ax=ax, importance_type='split')

# Modify font properties for labels, title, and tick labels
ax.set_xlabel(ax.get_xlabel(), fontname='Arial', fontsize=11)
ax.set_ylabel(ax.get_ylabel(), fontname='Arial', fontsize=11)
ax.set_title("Top 15 Features by Importance", fontname='Arial', fontsize=11)
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontname('Arial')
    item.set_fontsize(11)

plt.savefig("Top 15 Features by Importance Arial.png", bbox_inches='tight') # changed the name to avoid overwrite
plt.show() # show the plot

In [143]:
%%time 


#optuna.logging.set_verbosity(optuna.logging.WARNING)

import optuna, gc
gc.collect()

if os.path.isfile('N_Area_another_lgbm_with_val.pkl'):
    print("Found file !")
    study = joblib.load('N_Area_another_lgbm_with_val.pkl')
else:
    print("CREATING NEW FILE...")
    sampler = optuna.samplers.TPESampler(multivariate=True, warn_independent_sampling=False)
    study = optuna.create_study(directions=['maximize', 'maximize','maximize','maximize'], load_if_exists=True,sampler=sampler,pruner=optuna.pruners.MedianPruner(n_warmup_steps=30,n_startup_trials=20))
study.optimize(objective, timeout=1_000_000, gc_after_trial=True,
              show_progress_bar=True, n_jobs=1, callbacks=[logging_callback])
print("Finished working")
joblib.dump(study, 'N_Area_another_lgbm_with_val.pkl')

CREATING NEW FILE...


/home/20212358/.local/lib/python3.10/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-04-02 17:30:26,636] A new study created in memory with name: no-name-93d320ed-d0b5-496d-8611-b5b2dd4f4e58


   0%|          | 00:00/277:46:40

BACKUP GOES BRRRRR
[I 2025-04-02 17:30:27,042] Trial 0 finished with values: [0.7959183673469389, 0.8438801825034716, 0.7751937984496124, 0.5933333333333333] and parameters: {'boosting_type': 'rf', 'num_leaves': 326, 'max_depth': 746, 'objective': 'binary', 'learning_rate': 0.6397610099506099, 'n_estimators': 1389, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 77.21031970557576, 'feature_fraction': 0.31188381662988024, 'feature_fraction_bynode': 0.5003921158528095, 'extra_trees': True, 'lambda_l1': 37.39303874316047, 'lambda_l2': 64.43214141559666, 'linear_lambda': 90.93723222910305, 'drop_rate': 0.9319821437635308, 'max_drop': 351, 'skip_drop': 0.9696737198988122, 'uniform_drop': False, 'top_rate': 0.21067519581036181, 'other_rate': 0.20034064873084406, 'min_data_per_group': 1773, 'max_cat_threshold': 987, 'cat_l2': 22.2093686903579, 'cat_smooth': 64.01856182981004, 'top_k': 1358, 'cegb_tradeoff': 23.01054

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


...

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 18:18:17,717] Trial 624 finished with values: [0.4387755102040816, 0.43929775838127355, 0.5, 0.5] and parameters: {'boosting_type': 'dart', 'num_leaves': 337, 'max_depth': 818, 'objective': 'binary', 'learning_rate': 0.16843537569413758, 'n_estimators': 19888, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 44.99007472424334, 'feature_fraction': 0.8903877703578533, 'feature_fraction_bynode': 0.32987920620611866, 'extra_trees': True, 'lambda_l1': 8.233486547943059, 'lambda_l2': 84.95572344074874, 'linear_lambda': 27.377196769655487, 'drop_rate': 0.13255884154640218, 'max_drop': 118, 'skip_drop': 0.07334865001325341, 'uniform_drop': True, 'top_rate': 0.2362496051392971, 'other_rate': 0.0104590595258916, 'min_data_per_group': 1604, 'max_cat_threshold': 416, 'cat_l2': 31.144470889180003, 'cat_smooth': 88.02934501857352, 'top_k': 1837, 'cegb_tradeoff': 9.029151758654898, 'scale_pos_weight': 53.81990

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 18:18:20,805] Trial 625 finished with values: [0.5, 0.5, 0.5, 0.5] and parameters: {'boosting_type': 'gbdt', 'num_leaves': 148, 'max_depth': 539, 'objective': 'binary', 'learning_rate': 0.24741246761062882, 'n_estimators': 23858, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 18.368787621899997, 'feature_fraction': 0.6516023791606351, 'feature_fraction_bynode': 0.20104382394132184, 'extra_trees': True, 'lambda_l1': 9.404762903336398, 'lambda_l2': 54.93707034375489, 'linear_lambda': 84.77644640761815, 'drop_rate': 0.060770833780242284, 'max_drop': 676, 'skip_drop': 0.9417042694028889, 'uniform_drop': False, 'top_rate': 0.46982508338557005, 'other_rate': 0.24561156025567896, 'min_data_per_group': 14, 'max_cat_threshold': 948, 'cat_l2': 26.33093172680214, 'cat_smooth': 58.14170459306501, 'top_k': 1651, 'cegb_tradeoff': 68.23130027469347, 'scale_pos_weight': 10.039103204885187, 'sigmoid': 2.4838449

KeyboardInterrupt: 

In [ ]:
print(1/0)
#CHECK TRIAL 1580, 1709, 1845, 4669, 9338

#after second december 2024
#check 643 (0.17,0.94);  765 (0.16, 1.0);  801 (0.154, 1.0);

In [144]:
study = joblib.load('N_Area_another_lgbm_with_val.pkl')



In [145]:
# trial_number = 4669
# specific_trial = next((t for t in study.trials if t.number == trial_number), None)

# if specific_trial is not None:
#     print(f"Trial #{specific_trial.number}")
#     print("Parameters:", specific_trial.params)
#     print("Value:", specific_trial.values)
# else:
#     print(f"Trial #{trial_number} not found.")

In [146]:
print('Number of finished trials:', len(study.trials))
try:
    print('Best trial:', study.best_trial.params)
except Exception:
    pass

Number of finished trials: 626


In [147]:
try:
    Best_trial = study.best_trial.params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)
except Exception:
    Best_trial = study.best_trials[0].params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)

{'boosting_type': 'rf', 'num_leaves': 2237, 'max_depth': 660, 'objective': 'binary', 'learning_rate': 0.7758148922129062, 'n_estimators': 2662, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 92.15592306918016, 'feature_fraction': 0.13297535084146456, 'feature_fraction_bynode': 0.3955036705747294, 'extra_trees': False, 'lambda_l1': 45.84276381600275, 'lambda_l2': 48.776608970376316, 'linear_lambda': 49.610698473738545, 'drop_rate': 0.07863893437928382, 'max_drop': 935, 'skip_drop': 0.26227860512049955, 'uniform_drop': True, 'top_rate': 0.46621884740611774, 'other_rate': 0.3133873923545648, 'min_data_per_group': 49, 'max_cat_threshold': 399, 'cat_l2': 31.0935188867366, 'cat_smooth': 98.728766717671, 'top_k': 1961, 'cegb_tradeoff': 57.597486625282514, 'scale_pos_weight': 4.913082772611697, 'sigmoid': 78.3518785719258, 'max_bin': 665, 'n_jobs': -1, 'verbose': -1}


In [148]:
for i in range(len(study.best_trials)):
    print(study.best_trials[i].values)

[0.8520408163265306, 0.9190636778416981, 0.878038979304802, 0.7625]
[0.846938775510204, 0.9287839714342392, 0.878038979304802, 0.7380952380952381]
[0.8418367346938775, 0.9246181313231501, 0.8555689102564102, 0.8571428571428571]
[0.8367346938775512, 0.9236262646300337, 0.886491935483871, 0.7333333333333334]
[0.8520408163265306, 0.9293790914501091, 0.8759240759240758, 0.7333333333333334]
[0.8367346938775512, 0.929974211465979, 0.8732394366197183, 0.7916666666666667]
[0.8469387755102041, 0.9295774647887325, 0.878038979304802, 0.7046783625730995]
[0.8418367346938775, 0.9309660781590954, 0.8982371794871795, 0.7046783625730995]
[0.8418367346938775, 0.9331481848839516, 0.8100783874580066, 0.7625]
[0.826530612244898, 0.9345367982543147, 0.8436447811447811, 0.8571428571428571]
[0.8418367346938775, 0.9309660781590954, 0.8602985074626865, 0.7916666666666667]
[0.8469387755102041, 0.9260067446935132, 0.8594401429422275, 0.7620320855614974]
[0.8571428571428572, 0.9309660781590954, 0.8418812989921614

In [44]:
try:
    print(study.best_trial.value)
except Exception:
    print(study.best_trials[1].values)

[0.5791666666666666, 0.9885509227614491, 0.8928571428571428, 0.6966666666666668]


###### study.best_trials[1].params

In [165]:
%%capture

"""
try:
    best_model = lgb.LGBMClassifier(**study.best_params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])
except Exception:
    print("Using alternative")
    best_model = lgb.LGBMClassifier(**study.best_trials[0].params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])

"""

best_model = lgb.LGBMClassifier(**study.best_trials[2].params)
#best_model = lgb.LGBMClassifier(**specific_trial.params)
best_model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])

In [166]:
# best_model = cb.CatBoostClassifier(**study.best_params,
#                                 ).fit(X, Y, verbose=False, early_stopping_rounds=500)

In [167]:
best_model.predict(X_test)

array([ True,  True,  True,  True, False, False,  True,  True, False,
       False, False, False,  True,  True,  True, False, False, False,
       False,  True, False, False, False,  True,  True, False,  True,
        True])

In [168]:
predicted_train = best_model.predict(X_train)
probabilities_train = best_model.predict_proba(X_train)
predicted_test = best_model.predict(X_test)
probabilities_test = best_model.predict_proba(X_test)

#FOR VALIDATION
predicted_val = best_model.predict(X_val)
probabilities_val = best_model.predict_proba(X_val)

In [169]:
y_test.nunique()

2

In [170]:
from sklearn.metrics import balanced_accuracy_score

display(balanced_accuracy_score(y_test, predicted_test))
display(balanced_accuracy_score(y_train, predicted_train))

0.8571428571428571

0.852112676056338

In [171]:
# FOR VALIDATION

display(balanced_accuracy_score(y_val, predicted_val))

0.75

In [175]:
import matplotlib.pyplot as plt
from sklearn import metrics

# Assuming y_train, probabilities_train, y_test, probabilities_test are defined
plt.close('all')
# Calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:, 1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:, 1])

# Obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
print(train_roc_auc, test_roc_auc)

# Make the plot with Arial font and size 11
fig, ax = plt.subplots(figsize=[5, 5])
ax.set_title('Receiver Operating Characteristic', fontname='Arial', fontsize=11)
ax.plot(train_fpr, train_tpr, 'b', label='Train AUC = %0.2f' % train_roc_auc)
ax.plot(test_fpr, test_tpr, 'g', label='Test AUC = %0.2f' % test_roc_auc)
ax.legend(loc='lower right', prop={'family': 'Arial', 'size': 11})
ax.plot([0, 1], [0, 1], 'r--')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_ylabel('True Positive Rate', fontname='Arial', fontsize=11)
ax.set_xlabel('False Positive Rate', fontname='Arial', fontsize=11)
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontname('Arial')
    item.set_fontsize(11)

plt.savefig('another_lgbm_with_val_arial.png')  # Change the filename to avoid overwrite
plt.show() # show the plot

0.9246181313231501 0.8418367346938775


In [61]:
#-------------------

In [62]:
#together with Koen
bestClassyficationModel=RandomForestClassifier(**bestParameters)
bestClassyficationModel.fit(X_train, y_train)

NameError: name 'RandomForestClassifier' is not defined

In [135]:
predicted_train = bestClassyficationModel.predict(X_train)
probabilities_train = bestClassyficationModel.predict_proba(X_train)
predicted_test = bestClassyficationModel.predict(X_test)
probabilities_test = bestClassyficationModel.predict_proba(X_test)

NameError: name 'bestClassyficationModel' is not defined

Step 5.7 :Print out accuracy metrics

In [ ]:
print ('\nTrain Accuracy:', metrics.accuracy_score(y_train, predicted_train))
print ('Test Accuracy:', metrics.accuracy_score(y_test, predicted_test))


Step 5.8: plot the AUC curve for training and test data

In [ ]:
# calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
#obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
#make the plot
fig, ax = plt.subplots(figsize=[5,5])
plt.title('Receiver Operating Characteristic')
plt.plot(train_fpr, train_tpr, 'b', label = 'Train AUC = %0.2f' % train_roc_auc)
plt.plot(test_fpr, test_tpr, 'g', label = 'Test AUC = %0.2f' % test_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Step 5.9: Plot 10 most important features for the model

In [ ]:
plot_importance(bestClassyficationModel,max_num_features=20,xlabel='Features weight');    
plt.show()

### Step 6: Use SHAP module to explain predicted model

##### Step 6.1: Show which features contribute to prediction of particular surface

In [ ]:
explainerDefinition=explainer(dataForModel=X,
          model=bestClassyficationModel,
          hit=1)

##### Step 6.2: Show SHAP values for every important feature and every predicted surface Plot explanation https://shap.readthedocs.io/en/latest/

In [ ]:
shap_values = explainerDefinition.shap_values(X)
shap.summary_plot(shap_values, X)

##### Step 6.3: Draw a scatter plot with features of interest

Set the feature of interest for the x- and y-axis

In [ ]:
xFeature="Kamiel_FCP"
yFeature='InsCirclRadius_median'

In [ ]:
fig, ax = plt.subplots(figsize=[10,10])

colors = {0:'orange', 1:'blue'}

binned=X
binned["Class"]=Y

grouped = binned.groupby("Class")

for key, group in grouped:
    group.plot(ax=ax, kind='scatter', 
               x=xFeature, 
               y=yFeature, 
               label=key, color=colors[key])

plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# 1. Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 2. Scale Features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# 3. Hyperparameter Search Space Refinement
searchParameters = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.0001, 0.001, 0.01],
    'n_estimators': [100, 250, 500],
    'reg_alpha': [0.01, 0.1, 1],
    'reg_lambda': [0.01, 0.1, 1]
}

# 4. Updated Fitting Parameters
fittingParameters = {
    "early_stopping_rounds": 50,
    "eval_metric": "auc",
    "eval_set": [(X_test_scaled, y_test)]  # Using validation set for early stopping
}

# 5. Classification Model
classificationModel = XGBClassifier(objective='binary:logistic', random_state=1)

# 6. Randomized Search with Cross-Validation
random_search = RandomizedSearchCV(
    estimator=classificationModel,
    param_distributions=searchParameters,
    n_iter=50,  # Reduce iterations for quicker testing; increase later
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=10,
    random_state=1
)

# 7. Fit the Model
searchModel = random_search.fit(X_resampled_scaled, y_resampled, **fittingParameters)

# 8. Best Parameters
bestParameters = searchModel.best_estimator_.get_params()
print("Best Parameters:", bestParameters)

# 9. Evaluate Model on Test Data
y_pred = searchModel.predict(X_test_scaled)
test_auc = roc_auc_score(y_test, y_pred)
train_auc = roc_auc_score(y_resampled, searchModel.predict(X_resampled_scaled))

print(f"Train AUC: {train_auc}")
print(f"Test AUC: {test_auc}")
